# Feature Engineering for Classification

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression
import datetime
import re
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn import linear_model
from sklearn.preprocessing import StandardScaler
%matplotlib inline

In [2]:
za_data = pd.read_csv("Classification/all_files_ZAv1.csv")

In [3]:
za_data.columns

Index(['trackAppearanceCount', 'artistAppearanceCount', 'rank', 'uri',
       'artist_names', 'track_name', 'source', 'peak_rank', 'previous_rank',
       'weeks_on_chart', 'streams', 'End Date', 'Week', 'Year', 'End Date Dt',
       'Seconds since Epoch', 'End Date Aggregated', 'ArtistCount',
       'main_artist', 'isTopTen', 'Artist and Track', 'rank difference',
       'Position over Time'],
      dtype='object')

In [4]:
len(list(za_data.columns))

23

In [5]:
za_data["End Date Dt"] = za_data["End Date Dt"].apply(lambda x : datetime.strptime(x, '%Y-%m-%d'))

In [6]:
za_data.shape

(47486, 23)

In [7]:
za_data_test = za_data[za_data['End Date Dt'] > datetime(2022, 5, 26)]

za_data_test_first_time = za_data_test[za_data_test["trackAppearanceCount"] == 0]
za_data_test_first_time_df = za_data_test_first_time[['main_artist', 'track_name', "trackAppearanceCount", "artistAppearanceCount", "rank", "peak_rank", 'previous_rank', 'weeks_on_chart', 'streams', 'ArtistCount', 'isTopTen', 'rank difference']]


za_data_test = za_data_test[za_data_test["trackAppearanceCount"] != 0]
za_data = za_data[za_data['End Date Dt'] <= datetime(2022, 5, 26)]

In [8]:
za_data_df = za_data[['main_artist', 'track_name', "trackAppearanceCount", "artistAppearanceCount", "rank", "peak_rank", 'previous_rank', 'weeks_on_chart', 'streams', 'ArtistCount', 'isTopTen', 'rank difference']]
za_data_test_df = za_data_test[['main_artist', 'track_name', "trackAppearanceCount", "artistAppearanceCount", "rank", "peak_rank", 'previous_rank', 'weeks_on_chart', 'streams', 'ArtistCount', 'isTopTen', 'rank difference']]

In [9]:
# za_data["End Date"].unique().tolist()

In [10]:
groupby_cols = ['main_artist', 'track_name']

In [11]:
za_data_test_first_time

,trackAppearanceCount,artistAppearanceCount,rank,uri,artist_names,track_name,source,peak_rank,previous_rank,weeks_on_chart,...,Year,End Date Dt,Seconds since Epoch,End Date Aggregated,ArtistCount,main_artist,isTopTen,Artist and Track,rank difference,Position over Time
2,0,0,128,spotify:track:3KnHZwV1StOTvtHwpK2gK1,"031CHOPPA, Madumane, Focalistic, Xduppy, Kingtone",Usenemali,031Choppa,128,-1,1,...,2022,2022-09-08,1.662595e+09,2022-09-01,1,031CHOPPA,0,"031CHOPPA, Madumane, Focalistic, Xduppy, Kingt...",0,0
3,0,0,168,spotify:track:4yaPPFIafaVaMRHZ5IHkbA,"1da Banton, Kizz Daniel, Tiwa Savage",No Wahala - Remix,Squareball Ent. Limited,38,170,30,...,2022,2022-09-29,1.664410e+09,2022-09-01,1,1da Banton,0,"1da Banton, Kizz Daniel, Tiwa Savage; No Wahal...",0,0
862,0,57,66,spotify:track:0OHBji0Hz8o7W1TPYtk31j,"A-Reece, Marcus Harvey",DREAMS,REVENGE CLUB RECORDS,66,-1,1,...,2022,2022-09-22,1.663805e+09,2022-09-01,1,A-Reece,0,"A-Reece, Marcus Harvey; DREAMS",0,0
1010,0,78,2,spotify:track:6PqdJb18ssWDcROvnsEqy6,"AKA, Nasty C",Lemons (Lemonade),Sound African Recordings,2,-1,1,...,2022,2022-09-22,1.663805e+09,2022-09-01,1,AKA,1,"AKA, Nasty C; Lemons (Lemonade)",0,0
1094,0,0,109,spotify:track:4ucXeGcKKzubgHb932T6sl,AV,Confession,AV,67,126,33,...,2022,2022-09-29,1.664410e+09,2022-09-01,1,AV,0,AV; Confession,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47451,0,1,157,spotify:track:2QWEMqQMJR1KDf6hDjJOs6,j-hope,Arson,BIGHIT MUSIC,157,-1,1,...,2022,2022-07-21,1.658362e+09,2022-07-01,1,j-hope,0,j-hope; Arson,0,0
47452,0,0,95,spotify:track:4XKXphKH76W4zNpkgFS5o2,j-hope,MORE,BIGHIT MUSIC,95,-1,1,...,2022,2022-07-07,1.657152e+09,2022-07-01,1,j-hope,0,j-hope; MORE,0,0
47461,0,0,121,spotify:track:3vXFxsPqMgw4SYP7fkWicM,"melvitto, Wande Coal",Gentility,melvitto under exclusive license to Amuseio AB,12,117,24,...,2022,2022-09-29,1.664410e+09,2022-09-01,1,melvitto,0,"melvitto, Wande Coal; Gentility",0,0
47463,0,0,165,spotify:track:4ZxL5Fi5K0nkyTmsbwPdsj,rum.gold,Call It What You Want,Leola LLC,165,-1,1,...,2022,2022-06-16,1.655338e+09,2022-06-01,1,rum.gold,0,rum.gold; Call It What You Want,0,0


### 0. Entry Position into Chart

In [12]:
def findingtheentrypos(za_data):
    
    groupednum_songs = za_data.groupby(groupby_cols)["End Date"].agg("min")
    groupednum_songs_df = pd.DataFrame(groupednum_songs).reset_index()
    groupednum_songs_df.columns = groupby_cols + ["Artist Song Start Date"]
    
    groupednum_songs_df = groupednum_songs_df.drop_duplicates()
    groupednum_songs_df.shape
    
    groupednum_songs_ls = groupednum_songs_df.values.tolist()
    
    thefirstones = []
    for artsongls in groupednum_songs_ls:
        theartist = artsongls[0]
        thetrack = artsongls[1]
        thestartdate = artsongls[2]
        thefirstones.append(za_data[(za_data["main_artist"] == theartist) & (za_data["track_name"] == thetrack) & (za_data["End Date"] == thestartdate)])
        
    first_track_position_df = pd.concat(thefirstones)
    
    first_track_position_df_relevant = first_track_position_df[["main_artist", "track_name", "rank", "End Date"]]
    first_track_position_df_relevant.columns = ["main_artist", "track_name", "entry rank", "End Date"]
    
    groupednum_songs = za_data.groupby(groupby_cols)["End Date"].agg("count")
    groupednum_songs_df = pd.DataFrame(groupednum_songs).reset_index()
    groupednum_songs_df.columns = groupby_cols + ["Artist Song Count"]
    
    return groupednum_songs_df, first_track_position_df_relevant

### 1. Streams

In [13]:
def findtheMinMaxMeanStreams(za_data):
    groupedmax_streams = za_data.groupby(groupby_cols)["streams"].agg("max")
    groupedmax_streams_df = pd.DataFrame(groupedmax_streams).reset_index()
    groupedmax_streams_df.columns = groupby_cols + ["Max Streams per Song"]
    
    groupedmin_streams = za_data.groupby(groupby_cols)["streams"].agg("min")
    groupedmin_streams_df = pd.DataFrame(groupedmin_streams).reset_index()
    groupedmin_streams_df.columns = groupby_cols + ["Min Streams per Song"]
    
    groupedsum_streams = za_data.groupby(groupby_cols)["streams"].agg("sum")
    groupedsum_streams_df = pd.DataFrame(groupedsum_streams).reset_index()
    groupedsum_streams_df.columns = groupby_cols + ["Total Streams per Song"]

    groupedmean_streams = za_data.groupby(groupby_cols)["streams"].agg("mean")
    groupedmean_streams_df = pd.DataFrame(groupedmean_streams).reset_index()
    groupedmean_streams_df.columns = groupby_cols + ["Mean Streams per Song"]
    
    return groupedmax_streams_df, groupedmin_streams_df, groupedsum_streams_df, groupedmean_streams_df

### 2. Positions

In [14]:
def findtheMinMaxPositions(za_data):
    groupedpos_streams = za_data.groupby(groupby_cols)["rank"].agg("min")
    groupedpos_streams_df = pd.DataFrame(groupedpos_streams).reset_index()
    groupedpos_streams_df.columns = groupby_cols + ["Min Position per Song"]

    groupedposmax_streams = za_data.groupby(groupby_cols)["rank"].agg("max")
    groupedposmax_streams_df = pd.DataFrame(groupedposmax_streams).reset_index()
    groupedposmax_streams_df.columns = groupby_cols + ["Max Position per Song"]
    
    return groupedpos_streams_df, groupedposmax_streams_df

### 3. Dates

## Combine the dfs

In [15]:
def combinedataframes(groupedmax_streams_df, groupedmin_streams_df, groupedsum_streams_df, groupedmean_streams_df, groupedpos_streams_df, groupedposmax_streams_df, groupednum_songs_df):
    groupeddf = groupedmax_streams_df
    
    groupeddf["Min Streams per Song"] = groupedmin_streams_df["Min Streams per Song"]
    groupeddf["Total Streams per Song"] = groupedsum_streams_df["Total Streams per Song"]
    groupeddf["Mean Streams per Song"]= groupedmean_streams_df["Mean Streams per Song"]
    groupeddf["Min Position per Song"] = groupedpos_streams_df["Min Position per Song"]
    groupeddf["Max Position per Song"] = groupedposmax_streams_df["Max Position per Song"]
    groupeddf["Artist Song Count"] = groupednum_songs_df["Artist Song Count"]
    
    groupeddf["MaxMin Difference Position"] = groupeddf["Max Position per Song"] - groupeddf["Min Position per Song"]
    groupeddf["MaxMin Difference Streams"] = groupeddf["Max Streams per Song"] - groupeddf["Min Streams per Song"]
    
    groupeddf["MaxMin Ratio Position"] = groupeddf["Max Position per Song"] / groupeddf["Min Position per Song"]
    groupeddf["MaxMin Ratio Streams"] = groupeddf["Max Streams per Song"] / groupeddf["Min Streams per Song"]
    
    groupeddf.drop(["Max Position per Song"], inplace=True, axis=1)
    
    groupeddf_startpos = groupeddf.merge(first_track_position_df_relevant[["main_artist", "track_name", "entry rank"]], left_on=["main_artist", "track_name"], right_on=["main_artist", "track_name"], how="inner")
    
    groupeddf_startpos["main_art_track"] = groupeddf_startpos["main_artist"] + groupeddf_startpos["track_name"]
    
    groupeddf_startpos_regress = groupeddf_startpos[['main_artist', "main_art_track", 'entry rank', 'Min Position per Song']].drop_duplicates()
    
    return groupeddf, groupeddf_startpos_regress
    

### THE KMEANS

In [16]:
def thekmeansfunc(za_data_df, groupeddf, groupeddf_startpos_regress, groupby_cols, filenname):
    grouped_za_data_df = za_data_df.merge(groupeddf, left_on=["main_artist", "track_name"], right_on=["main_artist", "track_name"], how="inner")
    
    train_groupeddf = grouped_za_data_df.drop(groupby_cols, axis=1)
    
    kmeans = KMeans(n_clusters=4, random_state=0).fit(train_groupeddf)
    
    allklabels = kmeans.labels_
    
    print(len(grouped_za_data_df), len(train_groupeddf))
    
    grouped_za_data_df["Labels"] = allklabels
    
    other_cols = ["Min Streams per Song", "Total Streams per Song", "Mean Streams per Song", "Min Position per Song", "Artist Song Count", "MaxMin Ratio Position", "MaxMin Ratio Streams", "MaxMin Difference Position", "MaxMin Difference Streams"]
    
    for col in other_cols:
        grouped_za_data_df[col] = groupeddf[col]
        
    grouped_za_data_df.to_csv("grouped_za_data_df" + filenname +".csv", index=False)
    
    grouped_za_data_df["main_art_track"] = grouped_za_data_df["main_artist"] + grouped_za_data_df["track_name"]
    
    grouped_za_data_df = grouped_za_data_df.merge(groupeddf_startpos_regress, left_on=["main_art_track"], right_on=["main_art_track"], how="left")
    
    grouped_za_data_df.columns = ['main_artist', 'track_name', 'trackAppearanceCount', 'artistAppearanceCount', 'rank', 'peak_rank', 'previous_rank','weeks_on_chart', 'streams', 'ArtistCount', 'isTopTen', 'rank difference', 'Max Streams per Song', 'Min Streams per Song', 'Total Streams per Song', 'Mean Streams per Song', 'Min Position per Song', 'Artist Song Count','MaxMin Difference Position', 'MaxMin Difference Streams','MaxMin Ratio Position', 'MaxMin Ratio Streams', 'Labels', 'main_art_track', 'main_artist_y', 'entry rank', 'Min Position per Song_y']
    
    grouped_za_data_df.drop(['main_artist_y', 'Min Position per Song_y'], axis=1, inplace=True)
    
    labels_kmeans = grouped_za_data_df['Labels'].unique().tolist()
    
    grouped_za_data_df = grouped_za_data_df.dropna()
    
    print(grouped_za_data_df.shape)
    grouped_za_data_df = grouped_za_data_df.drop_duplicates()
    grouped_za_data_df.to_csv("grouped_za_data_df.csv", index=False)

    
    return grouped_za_data_df

### Linear Regression

In [49]:
def theLinearRegressionFuncFewGrouped(grouped_za_data_df, grouped_za_test_data_df, thelabelsmade):
    for alabel in thelabelsmade:

        pred_df_one = grouped_za_data_df[grouped_za_data_df["Labels"] == alabel]

#         pred_df_one = pred_df_one.drop(["main_artist", "track_name", "main_art_track"], axis=1)

        # Build linear regression model using TV and Radio as predictors
        # Split data into predictors X and output Y
#         predictors = list(pred_df_one.columns)
#         predictors = ['Artist Song Count', 'Total Streams per Song', 'Min Streams per Song', 'MaxMin Difference Position', 'MaxMin Ratio Streams']
        
        X = pred_df_one
        X = X.drop(['Min Position per Song'],axis=1)
        y = pred_df_one['Min Position per Song']

        # Initialise and fit model
        lm = LinearRegression()
        model = lm.fit(X, y)

        grouped_za_test_data_df_few = grouped_za_test_data_df[['Min Position per Song', 'Artist Song Count', 'Total Streams per Song', 'Min Streams per Song', 'MaxMin Difference Position', 'MaxMin Ratio Streams', "Labels"]]
        y = grouped_za_test_data_df_few['Min Position per Song']
        grouped_za_test_data_df_few = grouped_za_test_data_df_few.drop(['Min Position per Song'], axis=1)
        # grouped_za_test_data_df_minus = grouped_za_test_data_df.drop(["main_artist", "track_name", "main_art_track",'Min Position per Song'], axis=1)
        all_preds = model.predict(grouped_za_test_data_df_few)

        fewer_columns_mean_squared_error = mean_squared_error(y, all_preds)
        fewer_columns_r2_score = r2_score(y, all_preds)

        print(alabel, "Mean squared error: %.2f" % fewer_columns_mean_squared_error)
        # The coefficient of determination: 1 is perfect prediction
        print(alabel, "Coefficient of determination: %.2f" % fewer_columns_r2_score)
        
    return model

In [18]:
def theLinearRegressionFuncAllGrouped(grouped_za_data_df, thelabelsmade):
    for alabel in thelabelsmade:

        pred_df_one = grouped_za_data_df[grouped_za_data_df["Labels"] == alabel]

        pred_df_one = pred_df_one.drop(["main_artist", "track_name", "main_art_track"], axis=1)

        predictors = list(pred_df_one.columns)
        X = pred_df_one[predictors]
        X = X.drop(['Min Position per Song'],axis=1)
        y = pred_df_one['Min Position per Song']

        # Initialise and fit model
        lm = LinearRegression()
        model = lm.fit(X, y)

        print(f'alpha = {model.intercept_}')
        print(f'betas = {model.coef_}')

        all_preds = model.predict(X)

#         print("Mean squared error: %.2f" % mean_squared_error(y, all_preds))
#         # The coefficient of determination: 1 is perfect prediction
#         print("Coefficient of determination: %.2f" % r2_score(y, all_preds))
        
        return model

In [19]:
def theLinearRegressionFuncAll(grouped_za_data_df):
    pred_df_one = grouped_za_data_df

    pred_df_one = pred_df_one.drop(["main_artist", "track_name", "main_art_track"], axis=1)

    predictors = list(pred_df_one.columns)
    X = pred_df_one[predictors]
    X = X.drop(['Min Position per Song'],axis=1)
    y = pred_df_one['Min Position per Song']

    # Initialise and fit model
    lm = LinearRegression()
    model = lm.fit(X, y)

    print(f'alpha = {model.intercept_}')
    print(f'betas = {model.coef_}')

    all_preds = model.predict(X)

#     print("Mean squared error: %.2f" % mean_squared_error(y, all_preds))
#     # The coefficient of determination: 1 is perfect prediction
#     print("Coefficient of determination: %.2f" % r2_score(y, all_preds))
    
    return model

# Call the functions

In [20]:
scaler = StandardScaler()

In [21]:
some_artists_of_interest = ["6LACK", "9umba", "BNXN fka Buju", "Black Coffee", "Big Zulu"]
groupednum_songs_df, first_track_position_df_relevant = findingtheentrypos(za_data)
groupedmax_streams_df, groupedmin_streams_df, groupedsum_streams_df, groupedmean_streams_df = findtheMinMaxMeanStreams(za_data)
groupedpos_streams_df, groupedposmax_streams_df = findtheMinMaxPositions(za_data)
groupeddf, groupeddf_startpos_regress = combinedataframes(groupedmax_streams_df, groupedmin_streams_df, groupedsum_streams_df, groupedmean_streams_df, groupedpos_streams_df, groupedposmax_streams_df, groupednum_songs_df)
filenname = "zaall"

In [22]:
grouped_za_data_df = thekmeansfunc(za_data_df, groupeddf, groupeddf_startpos_regress, groupby_cols, filenname)
thelabelsmade = grouped_za_data_df["Labels"].unique().tolist()

43686 43686
(3797, 25)


In [23]:
some_artists_of_interest = ["6LACK", "9umba", "BNXN fka Buju", "Black Coffee", "Big Zulu"]
groupednum_songs_df, first_track_position_df_relevant = findingtheentrypos(za_data_test)
groupedmax_streams_df, groupedmin_streams_df, groupedsum_streams_df, groupedmean_streams_df = findtheMinMaxMeanStreams(za_data_test)
groupedpos_streams_df, groupedposmax_streams_df = findtheMinMaxPositions(za_data_test)
groupeddf, groupeddf_startpos_regress = combinedataframes(groupedmax_streams_df, groupedmin_streams_df, groupedsum_streams_df, groupedmean_streams_df, groupedpos_streams_df, groupedposmax_streams_df, groupednum_songs_df)
filenname = "zatest"
grouped_za_test_data_df = thekmeansfunc(za_data_test_df, groupeddf, groupeddf_startpos_regress, groupby_cols, filenname)
thelabelsmade_test = grouped_za_data_df["Labels"].unique().tolist()

3336 3336
(397, 25)


In [24]:
grouped_za_test_data_df.columns

Index(['main_artist', 'track_name', 'trackAppearanceCount',
       'artistAppearanceCount', 'rank', 'peak_rank', 'previous_rank',
       'weeks_on_chart', 'streams', 'ArtistCount', 'isTopTen',
       'rank difference', 'Max Streams per Song', 'Min Streams per Song',
       'Total Streams per Song', 'Mean Streams per Song',
       'Min Position per Song', 'Artist Song Count',
       'MaxMin Difference Position', 'MaxMin Difference Streams',
       'MaxMin Ratio Position', 'MaxMin Ratio Streams', 'Labels',
       'main_art_track', 'entry rank'],
      dtype='object')

In [25]:
grouped_za_data_df.columns

Index(['main_artist', 'track_name', 'trackAppearanceCount',
       'artistAppearanceCount', 'rank', 'peak_rank', 'previous_rank',
       'weeks_on_chart', 'streams', 'ArtistCount', 'isTopTen',
       'rank difference', 'Max Streams per Song', 'Min Streams per Song',
       'Total Streams per Song', 'Mean Streams per Song',
       'Min Position per Song', 'Artist Song Count',
       'MaxMin Difference Position', 'MaxMin Difference Streams',
       'MaxMin Ratio Position', 'MaxMin Ratio Streams', 'Labels',
       'main_art_track', 'entry rank'],
      dtype='object')

##### Calling Linear Regression

In [48]:
thelabelsmade

[1, 2, 0, 3]

In [50]:
grouped_za_data_df_few = grouped_za_data_df[['Min Position per Song', 'Artist Song Count', 'Total Streams per Song', 'Min Streams per Song', 'MaxMin Difference Position', 'MaxMin Ratio Streams', "Labels"]]
model = theLinearRegressionFuncFewGrouped(grouped_za_data_df_few, grouped_za_test_data_df, thelabelsmade)

# grouped_za_test_data_df_few = grouped_za_test_data_df[['Min Position per Song', 'Artist Song Count', 'Total Streams per Song', 'Min Streams per Song', 'MaxMin Difference Position', 'MaxMin Ratio Streams', "Labels"]]
# y = grouped_za_test_data_df_few['Min Position per Song']
# grouped_za_test_data_df_few = grouped_za_test_data_df_few.drop(['Min Position per Song'], axis=1)
# # grouped_za_test_data_df_minus = grouped_za_test_data_df.drop(["main_artist", "track_name", "main_art_track",'Min Position per Song'], axis=1)
# all_preds = model.predict(grouped_za_test_data_df_few)

# fewer_columns_mean_squared_error = mean_squared_error(y, all_preds)
# fewer_columns_r2_score = r2_score(y, all_preds)

# print("Mean squared error: %.2f" % fewer_columns_mean_squared_error)
# # The coefficient of determination: 1 is perfect prediction
# print("Coefficient of determination: %.2f" % fewer_columns_r2_score)

1 Mean squared error: 2408.87
1 Coefficient of determination: 0.29
2 Mean squared error: 2532.86
2 Coefficient of determination: 0.25
0 Mean squared error: 1939.86
0 Coefficient of determination: 0.43
3 Mean squared error: 2487.48
3 Coefficient of determination: 0.27


In [27]:
model = theLinearRegressionFuncAllGrouped(grouped_za_data_df, thelabelsmade)

y = grouped_za_test_data_df['Min Position per Song']
grouped_za_test_data_df_minus = grouped_za_test_data_df.drop(["main_artist", "track_name", "main_art_track",'Min Position per Song'], axis=1)
all_preds = model.predict(grouped_za_test_data_df_minus)


print("Mean squared error: %.2f" % mean_squared_error(y, all_preds))
# The coefficient of determination: 1 is perfect prediction
print("Coefficient of determination: %.2f" % r2_score(y, all_preds))

alpha = 164.93705103121508
betas = [-5.47170217e-01 -1.53850928e-02 -2.45040258e-02  3.57260315e-02
  1.33784934e-02  7.12537498e-01  1.56218957e-04  4.77939676e+00
  1.12780476e+01  4.00698512e-02 -8.50368777e-05 -1.26479264e-03
 -3.88802860e-07 -5.57156095e-04  1.61779326e-01 -7.44021787e-01
  2.62335979e-04 -6.02629117e-02 -3.46624201e+00  0.00000000e+00
  4.56773803e-03]
Mean squared error: 2169.01
Coefficient of determination: 0.36


In [28]:
model = theLinearRegressionFuncAll(grouped_za_data_df)

y = grouped_za_test_data_df['Min Position per Song']
grouped_za_test_data_df_minus = grouped_za_test_data_df.drop(["main_artist", "track_name", "main_art_track",'Min Position per Song'], axis=1)
all_preds = model.predict(grouped_za_test_data_df_minus)


print("Mean squared error: %.2f" % mean_squared_error(y, all_preds))
# The coefficient of determination: 1 is perfect prediction
print("Coefficient of determination: %.2f" % r2_score(y, all_preds))

alpha = 166.6872200867307
betas = [-4.05855515e-01 -3.29045684e-02  4.50867031e-03  2.47222857e-02
  1.78838569e-02  4.20318933e-01  6.71243664e-05  2.38388195e+00
  3.14379081e+00  9.29766025e-03 -6.92960108e-05 -1.29364379e-03
 -5.05606133e-07 -3.88821606e-04  1.29633706e-01 -7.08620821e-01
  1.72819205e-04 -6.32044790e-02 -2.89866219e+00 -8.63787101e-02
 -2.52155097e-02]
Mean squared error: 2620.24
Coefficient of determination: 0.23


In [35]:
grouped_za_data_df.columns

Index(['main_artist', 'track_name', 'trackAppearanceCount',
       'artistAppearanceCount', 'rank', 'peak_rank', 'previous_rank',
       'weeks_on_chart', 'streams', 'ArtistCount', 'isTopTen',
       'rank difference', 'Max Streams per Song', 'Min Streams per Song',
       'Total Streams per Song', 'Mean Streams per Song',
       'Min Position per Song', 'Artist Song Count',
       'MaxMin Difference Position', 'MaxMin Difference Streams',
       'MaxMin Ratio Position', 'MaxMin Ratio Streams', 'Labels',
       'main_art_track', 'entry rank'],
      dtype='object')

In [40]:
clf = linear_model.PoissonRegressor()


pred_df_one = grouped_za_data_df

y = pred_df_one['Min Position per Song']
X = pred_df_one.drop(["main_artist", "track_name", "main_art_track",'Min Position per Song'], axis=1)

clf.fit(X, y)

print(clf.score(X, y))

print(clf.coef_)

print(clf.intercept_)
grouped_za_test_data_df_one = grouped_za_test_data_df
y = grouped_za_test_data_df_one['Min Position per Song']
grouped_za_test_data_df_minusone = grouped_za_test_data_df_one.drop(["main_artist", "track_name", "main_art_track",'Min Position per Song'], axis=1)

all_preds = clf.predict(grouped_za_test_data_df_minusone)

print("Mean squared error: %.2f" % mean_squared_error(y, all_preds))
# The coefficient of determination: 1 is perfect prediction
print("Coefficient of determination: %.2f" % r2_score(y, all_preds))

0.0
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
4.468006787043089
Mean squared error: 3437.35
Coefficient of determination: -0.01


C:\Users\lkhum\anaconda3\envs\tfenviron\lib\site-packages\sklearn\_loss\glm_distribution.py:132: RuntimeWarning: divide by zero encountered in true_divide
  return -2 * (y - y_pred) / self.unit_variance(y_pred)
C:\Users\lkhum\anaconda3\envs\tfenviron\lib\site-packages\sklearn\linear_model\_glm\glm.py:43: RuntimeWarning: invalid value encountered in multiply
  temp = d1 * family.deviance_derivative(y, y_pred, weights)
C:\Users\lkhum\anaconda3\envs\tfenviron\lib\site-packages\sklearn\_loss\glm_distribution.py:315: RuntimeWarning: divide by zero encountered in true_divide
  dev = 2 * (xlogy(y, y/y_pred) - y + y_pred)


In [39]:
clf = linear_model.PoissonRegressor()

for alabel in thelabelsmade:
    pred_df_one = grouped_za_data_df[grouped_za_data_df["Labels"] == alabel]
    
    y = pred_df_one['Min Position per Song']
    X = pred_df_one.drop(["main_artist", "track_name", "main_art_track",'Min Position per Song'], axis=1)

    clf.fit(X, y)

    print(clf.score(X, y))

    print(clf.coef_)

    print(clf.intercept_)
    grouped_za_test_data_df_one = grouped_za_test_data_df[grouped_za_test_data_df["Labels"] == alabel]
    y = grouped_za_test_data_df_one['Min Position per Song']
    grouped_za_test_data_df_minusone = grouped_za_test_data_df_one.drop(["main_artist", "track_name", "main_art_track",'Min Position per Song'], axis=1)

    all_preds = clf.predict(grouped_za_test_data_df_minusone)

    print("Mean squared error: %.2f" % mean_squared_error(y, all_preds))
    # The coefficient of determination: 1 is perfect prediction
    print("Coefficient of determination: %.2f" % r2_score(y, all_preds))

-2.220446049250313e-16
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
4.467077829138427
Mean squared error: 3280.29
Coefficient of determination: -0.01
0.0
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
4.487869988292151
Mean squared error: 3462.00
Coefficient of determination: -0.09
0.0
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
4.468705149411813
Mean squared error: 3633.00
Coefficient of determination: -0.04
0.0
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
4.468051988944283
Mean squared error: 3555.27
Coefficient of determination: -0.02


C:\Users\lkhum\anaconda3\envs\tfenviron\lib\site-packages\sklearn\_loss\glm_distribution.py:132: RuntimeWarning: divide by zero encountered in true_divide
  return -2 * (y - y_pred) / self.unit_variance(y_pred)
C:\Users\lkhum\anaconda3\envs\tfenviron\lib\site-packages\sklearn\linear_model\_glm\glm.py:43: RuntimeWarning: invalid value encountered in multiply
  temp = d1 * family.deviance_derivative(y, y_pred, weights)
C:\Users\lkhum\anaconda3\envs\tfenviron\lib\site-packages\sklearn\_loss\glm_distribution.py:315: RuntimeWarning: divide by zero encountered in true_divide
  dev = 2 * (xlogy(y, y/y_pred) - y + y_pred)
C:\Users\lkhum\anaconda3\envs\tfenviron\lib\site-packages\sklearn\_loss\glm_distribution.py:132: RuntimeWarning: divide by zero encountered in true_divide
  return -2 * (y - y_pred) / self.unit_variance(y_pred)
C:\Users\lkhum\anaconda3\envs\tfenviron\lib\site-packages\sklearn\linear_model\_glm\glm.py:43: RuntimeWarning: invalid value encountered in multiply
  temp = d1 * fami

In [41]:
za_data_test_first_time

,trackAppearanceCount,artistAppearanceCount,rank,uri,artist_names,track_name,source,peak_rank,previous_rank,weeks_on_chart,...,Year,End Date Dt,Seconds since Epoch,End Date Aggregated,ArtistCount,main_artist,isTopTen,Artist and Track,rank difference,Position over Time
2,0,0,128,spotify:track:3KnHZwV1StOTvtHwpK2gK1,"031CHOPPA, Madumane, Focalistic, Xduppy, Kingtone",Usenemali,031Choppa,128,-1,1,...,2022,2022-09-08,1.662595e+09,2022-09-01,1,031CHOPPA,0,"031CHOPPA, Madumane, Focalistic, Xduppy, Kingt...",0,0
3,0,0,168,spotify:track:4yaPPFIafaVaMRHZ5IHkbA,"1da Banton, Kizz Daniel, Tiwa Savage",No Wahala - Remix,Squareball Ent. Limited,38,170,30,...,2022,2022-09-29,1.664410e+09,2022-09-01,1,1da Banton,0,"1da Banton, Kizz Daniel, Tiwa Savage; No Wahal...",0,0
862,0,57,66,spotify:track:0OHBji0Hz8o7W1TPYtk31j,"A-Reece, Marcus Harvey",DREAMS,REVENGE CLUB RECORDS,66,-1,1,...,2022,2022-09-22,1.663805e+09,2022-09-01,1,A-Reece,0,"A-Reece, Marcus Harvey; DREAMS",0,0
1010,0,78,2,spotify:track:6PqdJb18ssWDcROvnsEqy6,"AKA, Nasty C",Lemons (Lemonade),Sound African Recordings,2,-1,1,...,2022,2022-09-22,1.663805e+09,2022-09-01,1,AKA,1,"AKA, Nasty C; Lemons (Lemonade)",0,0
1094,0,0,109,spotify:track:4ucXeGcKKzubgHb932T6sl,AV,Confession,AV,67,126,33,...,2022,2022-09-29,1.664410e+09,2022-09-01,1,AV,0,AV; Confession,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47451,0,1,157,spotify:track:2QWEMqQMJR1KDf6hDjJOs6,j-hope,Arson,BIGHIT MUSIC,157,-1,1,...,2022,2022-07-21,1.658362e+09,2022-07-01,1,j-hope,0,j-hope; Arson,0,0
47452,0,0,95,spotify:track:4XKXphKH76W4zNpkgFS5o2,j-hope,MORE,BIGHIT MUSIC,95,-1,1,...,2022,2022-07-07,1.657152e+09,2022-07-01,1,j-hope,0,j-hope; MORE,0,0
47461,0,0,121,spotify:track:3vXFxsPqMgw4SYP7fkWicM,"melvitto, Wande Coal",Gentility,melvitto under exclusive license to Amuseio AB,12,117,24,...,2022,2022-09-29,1.664410e+09,2022-09-01,1,melvitto,0,"melvitto, Wande Coal; Gentility",0,0
47463,0,0,165,spotify:track:4ZxL5Fi5K0nkyTmsbwPdsj,rum.gold,Call It What You Want,Leola LLC,165,-1,1,...,2022,2022-06-16,1.655338e+09,2022-06-01,1,rum.gold,0,rum.gold; Call It What You Want,0,0


### Calling Linear Regression on first timers

In [31]:
some_artists_of_interest = ["6LACK", "9umba", "BNXN fka Buju", "Black Coffee", "Big Zulu"]

groupednum_songs_df, first_track_position_df_relevant = findingtheentrypos(za_data_test_first_time)
groupedmax_streams_df, groupedmin_streams_df, groupedsum_streams_df, groupedmean_streams_df = findtheMinMaxMeanStreams(za_data_test_first_time)
groupedpos_streams_df, groupedposmax_streams_df = findtheMinMaxPositions(za_data_test_first_time)
groupeddf, groupeddf_startpos_regress = combinedataframes(groupedmax_streams_df, groupedmin_streams_df, groupedsum_streams_df, groupedmean_streams_df, groupedpos_streams_df, groupedposmax_streams_df, groupednum_songs_df)
filenname = "za_test_first_time"
grouped_za_test_data_first_time_df = thekmeansfunc(za_data_test_first_time_df, groupeddf, groupeddf_startpos_regress, groupby_cols, filenname)
thelabelsmade_first_time_test = grouped_za_test_data_first_time_df["Labels"].unique().tolist()

464 464
(461, 25)


In [32]:
# model = theLinearRegressionFuncFewGrouped(grouped_za_data_df, thelabelsmade)

# y = grouped_za_test_data_first_time_df['Min Position per Song']
# grouped_za_test_data_first_time_df_minus = grouped_za_test_data_first_time_df.drop(["main_artist", "track_name", "main_art_track",'Min Position per Song'], axis=1)
# all_preds = model.predict(grouped_za_test_data_first_time_df_minus)

# print("Mean squared error: %.2f" % mean_squared_error(y, all_preds))
# # The coefficient of determination: 1 is perfect prediction
# print("Coefficient of determination: %.2f" % r2_score(y, all_preds))

model = theLinearRegressionFuncAllGrouped(grouped_za_data_df, thelabelsmade)

y = grouped_za_test_data_first_time_df['Min Position per Song']
grouped_za_test_data_first_time_df_minus = grouped_za_test_data_first_time_df.drop(["main_artist", "track_name", "main_art_track",'Min Position per Song'], axis=1)
all_preds = model.predict(grouped_za_test_data_first_time_df_minus)


print("Mean squared error: %.2f" % mean_squared_error(y, all_preds))
# The coefficient of determination: 1 is perfect prediction
print("Coefficient of determination: %.2f" % r2_score(y, all_preds))



model = theLinearRegressionFuncAll(grouped_za_data_df)

y = grouped_za_test_data_first_time_df['Min Position per Song']
grouped_za_test_data_first_time_df_minus = grouped_za_test_data_first_time_df.drop(["main_artist", "track_name", "main_art_track",'Min Position per Song'], axis=1)
all_preds = model.predict(grouped_za_test_data_first_time_df_minus)


print("Mean squared error: %.2f" % mean_squared_error(y, all_preds))
# The coefficient of determination: 1 is perfect prediction
print("Coefficient of determination: %.2f" % r2_score(y, all_preds))

alpha = 164.93705103121508
betas = [-5.47170217e-01 -1.53850928e-02 -2.45040258e-02  3.57260315e-02
  1.33784934e-02  7.12537498e-01  1.56218957e-04  4.77939676e+00
  1.12780476e+01  4.00698512e-02 -8.50368777e-05 -1.26479264e-03
 -3.88802860e-07 -5.57156095e-04  1.61779326e-01 -7.44021787e-01
  2.62335979e-04 -6.02629117e-02 -3.46624201e+00  0.00000000e+00
  4.56773803e-03]
Mean squared error: 3052.47
Coefficient of determination: 0.09
alpha = 166.6872200867307
betas = [-4.05855515e-01 -3.29045684e-02  4.50867031e-03  2.47222857e-02
  1.78838569e-02  4.20318933e-01  6.71243664e-05  2.38388195e+00
  3.14379081e+00  9.29766025e-03 -6.92960108e-05 -1.29364379e-03
 -5.05606133e-07 -3.88821606e-04  1.29633706e-01 -7.08620821e-01
  1.72819205e-04 -6.32044790e-02 -2.89866219e+00 -8.63787101e-02
 -2.52155097e-02]
Mean squared error: 2948.68
Coefficient of determination: 0.12


### Descriptions

In [33]:
for thelabel in labels_kmeans:
    number_ones = grouped_za_data_df[grouped_za_data_df['Labels'] == thelabel]
    print(number_ones.shape)
    x = number_ones['entry rank'].tolist()
    y = number_ones['Min Position per Song'].tolist()

    plt.title(thelabel)
    plt.xlabel('entry rank')
    plt.ylabel('Min Position per Song')
    plt.scatter(x, y)
    plt.show()

NameError: name 'labels_kmeans' is not defined

In [ ]:
the_labels_more = grouped_za_data_df["Labels"].unique().tolist()

In [ ]:
for alabel in the_labels_more:
    kmeans_df_zero = grouped_za_data_df[grouped_za_data_df["Labels"] == alabel]
    description = kmeans_df_zero.describe()

number of artists in label
number of top 10 songs in label


### do it at an artist level but maybe in groups so that you do not need a lot of data for everyone.

In [ ]:
# for theartist in some_artists_of_interest:
#     number_ones = groupeddf_startpos_regress[groupeddf_startpos_regress['main_artist'] == theartist]

#     x = number_ones['entry rank'].tolist()
#     y = number_ones['Min Position per Song'].tolist()

#     plt.scatter(x, y)
#     plt.title(theartist)
#     plt.xlabel('entry rank')
#     plt.ylabel('Min Position per Song')
#     plt.show()

In [ ]:
# number_ones = groupeddf_startpos_regress[groupeddf_startpos_regress['main_artist'] == "salem ilese"]
# print(number_ones.shape)
# x = number_ones['entry rank'].tolist()
# y = number_ones['Min Position per Song'].tolist()

# plt.scatter(x, y)
# plt.show()